In [40]:
import pandas 
import os
import pandas as pd
# import faiss
from uuid import uuid4
from langgraph_supervisor import create_supervisor
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_core.documents import Document
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_community.utilities import SQLDatabase
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate

%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv
load_dotenv()


db = SQLDatabase.from_uri("sqlite:///xinfinity_agent.db")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
llm = ChatOpenAI(model="gpt-4.1-nano-2025-04-14")
embeddings = OpenAIEmbeddings()

In [5]:

vector_store = FAISS.load_local(
    "vector_store_index", embeddings, allow_dangerous_deserialization=True
)
retriever_tool = create_retriever_tool(
    retriever= vector_store.as_retriever(),
    name='retriever_tool',
    description='For fetching any company related policies or guidelines information'
)


In [ ]:
system_message=""

In [20]:
rag_react_agent = create_react_agent(model= llm, 
                                        tools= [retriever_tool], 
                                        name= 'rag_agent_executor',
                                        prompt= "You are a RAG agent that answers user queries, you have access to the retriever tool for context")

In [ ]:
#"can i get full refund if i return xfinity box after 6 months. it is undamaged."

In [25]:
result = rag_react_agent.invoke({"input":{"messages": [("user", "can i get full refund if i return xfinity box after 6 months. it is undamaged.")]}})

In [31]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
#refund policy if i return xfinity box after 6 months. it is undamaged.

In [38]:
inputs={"messages": [("user", "how can i get full refund on my xfinity devices? ")]}

In [39]:

print_stream(rag_react_agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

how can i get full refund on my xfinity devices? 
================================== Ai Message ==================================
Name: rag_agent_executor
Tool Calls:
  retriever_tool (call_AjfT30zDFMPlsKDmBTg7MDp8)
 Call ID: call_AjfT30zDFMPlsKDmBTg7MDp8
  Args:
    query: Xfinity device return and refund policy
================================= Tool Message =================================
Name: retriever_tool

For avoidance of doubt, Xfinity Equipment remains Comcast-owned equipment, and Comcast retains title to all Xfinity Equipment, at all times, including but not limited to after payment of an Unreturned Equipment Fee. "Unreturned Equipment Fee" refers to a fee charged by Comcast to a subscriber for any unreturned Xfinity Equipment upon termination of the Services provided under this Agreement. The payment of an Unreturned Equipment Fee shall not result in a sale of, or the transfer of title to, a

In [41]:
workflow = create_supervisor(
        agents=[ rag_react_agent],
        model=llm,
        prompt=(
    
            "Role: You are an Xfinity customer support team supervisor, managing 2 kinds of tasks : 1. a customer billing sql analysis as well as a math expert. "
            "2. Answering questions related to Xfinitie's policy and guidelines. "
            "For customer billing realted questions, leverage the sql_agent_executor. "
            "For answering queries related to company's policies and guidelines, use rag_agent_executor to cascade rag_agent_executor's output to user without rephrasing it."
            "Avoid mentioning any agent name in your response."
            "Some questions may need you to interact with both tasks, do it accordingly."
        ),
        supervisor_name="Xfinity",
    )
app = workflow.compile(name="XfinityTest")


In [ ]:
inputs={"messages": [("user", "how can i get full refund on my xfinity devices? ")]}

In [46]:
final_resp= app.invoke({
    "messages": [
        {
            "role": "user",
            "content": 'how can i get full refund on my xfinity devices?'
        }
    ]
})